In [ ]:
#Interactive KPI viewer （UMTS/LTE）
#能够进行多小区或者站点查询，筛选小区名称及KPI名称显示图表

from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource,CDSView,GroupFilter,BooleanFilter

import os
import pymysql.cursors
import pandas as pd

os.chdir("d:\\Reference Documents\\Code\\Python\\WorkspaceforPython\\RNO_KPI_Chart")

connection = pymysql.connect(host='localhost',
                             user='Test',
                             password='1234',
                             db='sts',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
if connection:
    print("connect to database successfully")
    
try:
    with connection.cursor() as cursor:
        # Read a single record
        CELLNAME=input("CellName=")
        CELLNAME=CELLNAME.strip().upper()
        StartDate=input("StartDate(\"YYYY-MM-DD\")=")
        StartDate=StartDate.strip()
        sql_lte =("SELECT * FROM `daily_cell_lte_5.1` WHERE Time>= \'"+StartDate+"\'and CELLNAME=\'"+CELLNAME+"\' ORDER BY Time ASC")
        cursor.execute(sql_lte)
        result_lte = cursor.fetchall()
        sql_umts =("SELECT * FROM `daily_cell_sts_5.2` WHERE Time>= \'"+StartDate+"\'and CELLNAME=\'"+CELLNAME+"\' ORDER BY Time ASC")
        cursor.execute(sql_umts)
        result_umts = cursor.fetchall()
        if not (result_lte or result_umts):
            print("Query result is empty")
except:
    print("Error occured when querying")
finally:
    connection.close()
    
#umts_kpi=[....]
#lte_kpi=[.....]
#KPI=[]
if result_lte:
    df=pd.DataFrame(result_lte)
    #KPI=lte_kpi 
if result_umts:
    df=pd.DataFrame(result_umts)
    #KPI=umts_kpi

if result_lte or result_umts:
    source = ColumnDataSource(df)
    
    #for kpi in KPI   循环显示所有KPI图表
    x=source.data['Time']
    y=source.data['L-S1 SIG Setup Success Rate']
    

    # 创建一个新的图表，设置图标大小。
    #设置横坐标轴类型为datetime,参数列表参考bokeh.plotting figure
    p = figure(width=800, height=300,x_axis_type="datetime")
    
    #设置标题
    p.title.text="Replaced by KPI name"
    p.title.align="center"
    p.title.text_font_size="25px"

    # 添加图形，设置图例
    p.line(x, y, line_width=2,legend="Y")
    p.circle(x, y, legend="Y",size=7, color="firebrick", alpha=0.5)

    
    #设置坐标轴
    p.xaxis.axis_line_color = "lightgray"
    p.xaxis.major_tick_line_color="gray"
    
    p.yaxis.axis_line_color = "lightgray"
    p.yaxis.major_tick_line_color="gray"
    p.yaxis.minor_tick_line_color = None

    # output to jupyter notebook
    output_notebook()
    


In [ ]:
if result_lte or result_umts:
    source = ColumnDataSource(df)
    #创建筛选器
    filter_cellname=[True if cellname=="RS54ML1811" else False for cellname in source.data["CELLNAME"]]
    
    view=CDSView(source=source,filters=[BooleanFilter(filter_cellname)])
    
    tools = ["box_select", "hover", "reset"]
    p_test= figure(width=800, height=300,x_axis_type="datetime",tools=tools)
    p_test.circle(x="Time",y="L-eRAB Drop Rate_ALL",size=5, hover_color="red",source=source,view=view)
    
    output_notebook() 
    show(p_test)